In [48]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint 

In [8]:
mongo = MongoClient(port=27017)

In [9]:
print(mongo.list_database_names())

['Homework1', 'Project-3', 'admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [10]:
db = mongo['Project-3']

In [11]:
print(db.list_collection_names())

['video_games']


In [20]:
video_games = db['video_games']

In [13]:
print(db.video_games.find_one())

{'_id': ObjectId('65839489b3e6510dae7ccd06'), 'name': 'The Legend of Zelda: Ocarina of Time', 'platform': ' Nintendo 64', 'release_date': 'November 23, 1998', 'summary': 'As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages.', 'user_review': 9.1}


In [62]:
# Delete documents with user_review equal to "tbd"
result = video_games.delete_many({'user_review': 'tbd'})
print(f"{result.deleted_count} documents deleted")

# Now, retrieve the remaining documents and perform aggregation
pipeline = [
    {
        '$group': {
            '_id': '$name',
            'mean_user_review': {'$avg': '$user_review'}
        }
    }
]

result_after_deletion = list(video_games.aggregate(pipeline))

for item in result_after_deletion[:5]:
    pprint(item)

0 documents deleted
{'_id': 'inFamous 2', 'mean_user_review': 8.6}
{'_id': 'Trials of Mana', 'mean_user_review': 7.633333333333333}
{'_id': 'Rave Master', 'mean_user_review': 6.3}
{'_id': 'Hearthstone: Heroes of Warcraft', 'mean_user_review': 5.8}
{'_id': 'Boom Blox Bash Party', 'mean_user_review': 7.8}


In [63]:
documents_with_tbd = video_games.find({'user_review': 'tbd'})

# Print or process the matching documents
for document in documents_with_tbd:
    print(document)

In [64]:
cleaned_df = pd.DataFrame(result_after_deletion)
cleaned_df.head(15)

,_id,mean_user_review
0,inFamous 2,8.600000
1,Trials of Mana,7.633333
2,Rave Master,6.300000
3,Hearthstone: Heroes of Warcraft,5.800000
4,Boom Blox Bash Party,7.800000
5,Divekick,6.900000
6,Skyshine's Bedlam,5.000000
7,Arcanum: Of Steamworks and Magick Obscura,8.700000
8,Call of Duty: Advanced Warfare - Reckoning,5.700000
9,Krater,7.100000


In [65]:
cleaned_df['mean_user_review'] = cleaned_df['mean_user_review'].round(2)
cleaned_df = cleaned_df.rename(columns={'_id': 'Video Game'})
cleaned_df = cleaned_df.rename(columns={'mean_user_review': 'Average Rating'})

cleaned_df.head(15)

,Video Game,Average Rating
0,inFamous 2,8.60
1,Trials of Mana,7.63
2,Rave Master,6.30
3,Hearthstone: Heroes of Warcraft,5.80
4,Boom Blox Bash Party,7.80
5,Divekick,6.90
6,Skyshine's Bedlam,5.00
7,Arcanum: Of Steamworks and Magick Obscura,8.70
8,Call of Duty: Advanced Warfare - Reckoning,5.70
9,Krater,7.10


In [68]:
top_games = cleaned_df.sort_values(by='Average Rating', ascending=False)
top_games.head(15)

,Video Game,Average Rating
4805,GrimGrimoire,9.70
10141,Ghost Trick: Phantom Detective,9.70
2663,Metal Torrent,9.70
1183,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,9.70
9722,Tengami,9.70
6210,Crystar,9.60
3101,Diaries of a Spaceport Janitor,9.60
528,Astro's Playroom,9.30
10448,The Witcher 3: Wild Hunt,9.27
7130,Warcraft III: Reign of Chaos,9.20


In [69]:
bottom_games = cleaned_df.sort_values(by='Average Rating', ascending=True)
bottom_games.head(15)

,Video Game,Average Rating
9704,Madden NFL 21,0.35
706,Madden NFL 22,0.55
11020,Warcraft III: Reforged,0.60
9139,FIFA 20: Legacy Edition,0.70
10496,The Sims 4: Star Wars - Journey to Batuu,1.00
3492,When Ski Lifts Go Wrong,1.00
1142,FIFA 21,1.07
811,Call of Duty: Modern Warfare 3 - Defiance,1.20
3505,Fast & Furious: Showdown,1.30
7404,FIFA 20,1.30
